# Pivotting the table 

This notebook transforms raw logging data from `../data/describe.csv` into a format suitable for coding by RA. 

The original table has each row representing a single response in a trial, with multiple rows combining to represent a participant's responses. To facilitate agreement analysis, we pivot the table, where each row corresponds to a participant and each column to a response.

The processed data is stored in`../data/pivot.csv`, pending coding. The coded data is available in `../data/coded_data.xlsx`. 

For analyis we only consider valid participants. They are identified using information from `../data/survey.csv`; only students who completed the post-experiment survey are considered.

In [ ]:
import pandas as pd

In [ ]:
survey_df = pd.read_csv("../data/survey.csv")
describe_df = pd.read_csv("../data/describe.csv")

In [ ]:
uids = set(survey_df["uid"])
len(uids)

In [ ]:
describe_df = describe_df[describe_df["uid"].isin(uids)]

In [ ]:
# Sanity check, every study should have at least 30 trials
for uid in uids:
    n = len(describe_df[describe_df["uid"] == uid])
    assert n >= 30, f"{uid}, len = {n}"

In [ ]:
result = pd.pivot_table(
    describe_df,
    index=["uid", "name"],
    columns="object_id",  # we want to convert `object_id` values into its own columns
    values=["description", "seen"],  # columns to keep
    aggfunc="max",  # in our case, each student will only have one description/seen for an object, `max` will give us the value
)
result.head()

In [ ]:
# Flatten
result.columns = [
    "description_object_" + str(col[1])
    if col[0] == "description"
    else "seen_object_" + str(col[1])
    for col in result.columns
]
result.reset_index(inplace=True)
result.head()

In [ ]:
result.to_csv("../data/pivot.csv", index=False)